# Tutorial 1: Basic Usage of FastEstimator

In FastEstimator, there are 3 APIs that you need to know:

* Pipeline: takes care of loading data and preprocessing data, it usually happens on CPU.
* Network: responsible for trainable & differentiable model.
* Estimator: manages training loop.

Any deep learning implementation will follow the `Pipeline` -> `Network` -> `Estimator` process as illustrated below:

## Step 1: Pipeline

For in-memory data, `Pipeline` accepts a nested dictionary like {"train": {"x": x_train, "y": y_train}, "eval": {"x": x_eval, "y": y_eval}}. If valdiation data is not available, then `eval` key is not needed.

In [ ]:
import tensorflow as tf
import numpy as np
import fastestimator as fe
from fastestimator.pipeline.processing import Minmax

(x_train, y_train), (x_eval, y_eval) = tf.keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1) #adding channel dimension for convolution later
x_eval = np.expand_dims(x_eval, -1) #adding channel dimension for convolution later
data = {"train": {"x": x_train, "y": y_train}, "eval": {"x": x_eval, "y": y_eval}}
pipeline = fe.Pipeline(batch_size=32, data=data, ops=Minmax(inputs="x", outputs="x"))

## Step 2: Network

For Network architecture, users can choose one of the following:
* Define a custom network architecture using  `tf.keras.Model` `tf.keras.Sequential`.  
* Use existing architecture provided by `tf.keras.applications` or `fe.architecture`. 

In this tutorial, we are going to import a pre-defined LeNet architecture in [fastestimator.architecture.lenet](https://github.com/fastestimator/fastestimator/blob/master/fastestimator/architecture/lenet.py).

After defining the architecture, we can associate model definition with its own optimizer and expected loss name (default to be 'loss') to `FEModel`. 

In [ ]:
from fastestimator.architecture import LeNet
from fastestimator.network.model import FEModel
from fastestimator.network.model import ModelOp
from fastestimator.network.loss import SparseCategoricalCrossentropy

model = FEModel(model_def=LeNet, model_name="lenet", optimizer="adam", loss_name="loss")
network = fe.Network(ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), 
                          SparseCategoricalCrossentropy(y_pred="y_pred", y_true="y", outputs="loss")])

## Step 3: Estimator

`Estimator` takes both `pipeline` and `network` and combines them in training loop. Here's the basic usage of `Estimator`. 

In [ ]:
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=2)

## Start training

Since the data is already loaded in memory, the training is happening without any disk reading. 

In [ ]:
estimator.fit()